# IMPORTING LIBRARIES AND DATA

In [ ]:
!pip install yfinance
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report,confusion_matrix
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping
from xgboost import XGBClassifier

  Created wheel for yfinance: filename=yfinance-0.1.54-py2.py3-none-any.whl size=22409 sha256=15084a6f2bc289891ebf3d7a924afa6c45d40dde01b997250675c543d7e6de0b
  Stored in directory: /root/.cache/pip/wheels/f9/e3/5b/ec24dd2984b12d61e0abf26289746c2436a0e7844f26f2515c
Successfully built yfinance


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [12]:
data = yf.download("PETR4.SA", period='60d',  interval = "5m")

[*********************100%***********************]  1 of 1 completed


# PREPARING DAY TEST DF

In [14]:
df = data
df = df.drop(['High', 'Low', 'Adj Close', 'Open'],axis=1)

# Creating t-n columns for Close
for i in range(1,28):
    df['Close_Minus{}'.format(i)] = df['Close'].shift(i)

# Creating t-n columns for Volume    
for i in range(1,28):
    df['Volume_Minus{}'.format(i)] = df['Volume'].shift(i)

df = df.loc['2020-04-29 10:00:00-03:00':'2020-04-29 16:50:00-03:00']

scaler = MinMaxScaler()
df = scaler.fit_transform(df)

ValueError: ignored

# TRAIN/TEST MODEL

## Data preprocessing

In [ ]:
# Feature engineering
bvsp = data
bvsp = bvsp[['Close', 'Volume']]

# Creating t-n columns for Close
for i in range(1,28):
    bvsp['Close_Minus{}'.format(i)] = bvsp['Close'].shift(i)

# Creating t-n columns for Volume    
for i in range(1,28):
    bvsp['Volume_Minus{}'.format(i)] = bvsp['Volume'].shift(i)

bvsp = bvsp.loc[:'2020-04-28 16:50:00-03:00']

bvsp_ann = bvsp.dropna() # dropping NAs

bvsp_ann['Rise'] = 1
periods_ahead = 18 # 1 and a half hours
positive_variation = 1.02

for i in range(1,len(bvsp_ann)-periods_ahead):
    for j in range(1,periods_ahead):
        if bvsp_ann['Close'].iloc[i]/bvsp_ann['Close'].iloc[i+j] > positive_variation:
            bvsp_ann['Rise'].iloc[i] = 1
            break
        else:
            bvsp_ann['Rise'].iloc[i] = 0

bvsp_ann = bvsp_ann.iloc[:-periods_ahead]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

In [ ]:
bvsp_ann['Rise'].value_counts()

0    3427
1    1037
Name: Rise, dtype: int64

In [ ]:
# Splitting into subsets
X = bvsp_ann.drop('Rise', axis=1)
y = bvsp_ann['Rise']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)

# Scaling data
scaler = MinMaxScaler()
X_train= scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Creating Early Stop callback
early_stop = EarlyStopping(monitor='val_loss', mode='auto', verbose=1, patience=25)

# Building the model
model = Sequential()
model.add(Dense(units=25,activation='relu'))
model.add(Dense(units=24,activation='relu'))
model.add(Dense(units=23,activation='relu'))
model.add(Dense(units=22,activation='relu'))
model.add(Dense(units=1,activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
             metrics=['accuracy'])

# Fitting the model
model.fit(x=X_train, 
          y=y_train.values,
          epochs=2000, verbose=0,
         validation_data=(X_train,y_train.values),
         callbacks=[early_stop])

Epoch 1/2000
Epoch 2/2000
Epoch 3/2000
Epoch 4/2000
Epoch 5/2000
Epoch 6/2000
Epoch 7/2000
Epoch 8/2000
Epoch 9/2000
Epoch 10/2000
Epoch 11/2000
Epoch 12/2000
Epoch 13/2000
Epoch 14/2000
Epoch 15/2000
Epoch 16/2000
Epoch 17/2000
Epoch 18/2000
Epoch 19/2000
Epoch 20/2000
Epoch 21/2000
Epoch 22/2000
Epoch 23/2000
Epoch 24/2000
Epoch 25/2000
Epoch 26/2000
Epoch 27/2000
Epoch 28/2000
Epoch 29/2000
Epoch 30/2000
Epoch 31/2000
Epoch 32/2000
Epoch 33/2000
Epoch 34/2000
Epoch 35/2000
Epoch 36/2000
Epoch 37/2000
Epoch 38/2000
Epoch 39/2000
Epoch 40/2000
Epoch 41/2000
Epoch 42/2000
Epoch 43/2000
Epoch 44/2000
Epoch 45/2000
Epoch 46/2000
Epoch 47/2000
Epoch 48/2000
Epoch 49/2000
Epoch 50/2000
Epoch 51/2000
Epoch 52/2000
Epoch 53/2000
Epoch 54/2000
Epoch 55/2000
Epoch 56/2000
Epoch 57/2000
Epoch 58/2000
Epoch 59/2000
Epoch 60/2000
Epoch 61/2000
Epoch 62/2000
Epoch 63/2000
Epoch 64/2000
Epoch 65/2000
Epoch 66/2000
Epoch 67/2000
Epoch 68/2000
Epoch 69/2000
Epoch 70/2000
Epoch 71/2000
Epoch 72/2000
E

In [ ]:
# Getting loss and accuracy plot
model_loss = pd.DataFrame(model.history.history)
#model_loss.plot()

# Getting predictions metrics
predictions = model.predict_classes(X_test)
print(classification_report(y_test,predictions))
print(confusion_matrix(y_test,predictions))

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
              precision    recall  f1-score   support

           0       0.87      0.91      0.89      1030
           1       0.64      0.53      0.58       310

    accuracy                           0.82      1340
   macro avg       0.75      0.72      0.73      1340
weighted avg       0.81      0.82      0.82      1340

[[939  91]
 [146 164]]


# SHOW TIME

In [2]:
import pandas as pd
import numpy as np
from tensorflow import keras

model = keras.models.load_model(r'..\ml-financial-market\models')
df = pd.read_csv(r'..\ml-financial-market\data\interim\interim.csv')

portfolio = pd.DataFrame(index=df.index, columns=['position', 'stocks_owned', 'bank'])
portfolio['position'] = 'Sold'
portfolio['bank'] = 10000
portfolio['stocks_owned'] = 0
portfolio['Model Recommendation'] = 0
portfolio['close'] = df['close']
portfolio.head()

price_bought = 0

positive_variation = 0.02

model.predict_classes(np.array([df[i],]))[0][0]


KeyError: 0

In [10]:
df

,Unnamed: 0,close,volume,close_minus1,close_minus2,close_minus3,close_minus4,close_minus5,close_minus6,close_minus7,...,volume_minus19,volume_minus20,volume_minus21,volume_minus22,volume_minus23,volume_minus24,volume_minus25,volume_minus26,volume_minus27,Rise
0,27,23.170000,193600.0,23.150000,23.150000,23.129999,23.150000,23.190001,23.139999,23.170000,...,2473300.0,696700.0,456100.0,1094500.0,1822700.0,1190000.0,1002500.0,767500.0,936000.0,0
1,28,23.150000,263700.0,23.170000,23.150000,23.150000,23.129999,23.150000,23.190001,23.139999,...,1125500.0,2473300.0,696700.0,456100.0,1094500.0,1822700.0,1190000.0,1002500.0,767500.0,0
2,29,23.150000,235200.0,23.150000,23.170000,23.150000,23.150000,23.129999,23.150000,23.190001,...,908600.0,1125500.0,2473300.0,696700.0,456100.0,1094500.0,1822700.0,1190000.0,1002500.0,0
3,30,23.170000,2625600.0,23.150000,23.150000,23.170000,23.150000,23.150000,23.129999,23.150000,...,505200.0,908600.0,1125500.0,2473300.0,696700.0,456100.0,1094500.0,1822700.0,1190000.0,0
4,31,23.150000,194100.0,23.170000,23.150000,23.150000,23.170000,23.150000,23.150000,23.129999,...,1395800.0,505200.0,908600.0,1125500.0,2473300.0,696700.0,456100.0,1094500.0,1822700.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4617,4644,22.490000,467100.0,22.459999,22.480000,22.420000,22.400000,22.420000,22.330000,22.350000,...,2540600.0,3688100.0,2604400.0,1898600.0,1569100.0,889600.0,1515800.0,353600.0,151300.0,0
4618,4645,22.530001,1434500.0,22.490000,22.459999,22.480000,22.420000,22.400000,22.420000,22.330000,...,1239000.0,2540600.0,3688100.0,2604400.0,1898600.0,1569100.0,889600.0,1515800.0,353600.0,0
4619,4646,22.540001,1838500.0,22.530001,22.490000,22.459999,22.480000,22.420000,22.400000,22.420000,...,679200.0,1239000.0,2540600.0,3688100.0,2604400.0,1898600.0,1569100.0,889600.0,1515800.0,0
4620,4647,22.559999,823400.0,22.540001,22.530001,22.490000,22.459999,22.480000,22.420000,22.400000,...,667100.0,679200.0,1239000.0,2540600.0,3688100.0,2604400.0,1898600.0,1569100.0,889600.0,0


In [5]:
model.predict_classes(np.array([df[0],]))

KeyError: 0

In [ ]:
portfolio = pd.DataFrame(index=df.index, columns=['position', 'stocks_owned', 'bank'])
portfolio['position'] = 'Sold'
portfolio['bank'] = 10000
portfolio['stocks_owned'] = 0
portfolio['Model Recommendation'] = 0
portfolio['Close'] = df['Close']
portfolio.head()

price_bought = 0

for i in range(len(portfolio)-1):
	buy_or_not = model.predict_classes(np.array([dft[i],]))[0][0]
	portfolio['Model Recommendation'].iloc[i] = buy_or_not
	if portfolio['position'].iloc[i] == 'Sold':
		if buy_or_not == 1:
			portfolio['stocks_owned'].iloc[i+1] = int(portfolio['bank'].iloc[i]/portfolio['Close'].iloc[i])
			portfolio['bank'].iloc[i+1] = (portfolio['bank'].iloc[i] % portfolio['Close'].iloc[i])
			portfolio['position'].iloc[i+1] = 'Bought'
			price_bought = portfolio['Close'].iloc[i]
			print('Comprou por {}'.format(price_bought))
		else:
			portfolio['position'].iloc[i+1] = portfolio['position'].iloc[i]
			portfolio['bank'].iloc[i+1] = portfolio['bank'].iloc[i]
			portfolio['stocks_owned'].iloc[i+1] = portfolio['stocks_owned'].iloc[i]
			print('Não comprou')
	elif portfolio['Close'].iloc[i] < (0.95) * price_bought:
		portfolio['bank'].iloc[i+1] = portfolio['bank'].iloc[i] + portfolio['Close'].iloc[i] * portfolio['stocks_owned'].iloc[i]
		portfolio['stocks_owned'].iloc[i+1] = 0
		portfolio['position'].iloc[i+1] = 'Sold'
		print("Vendeu por {}".format(portfolio['Close'].iloc[i]))
	elif portfolio['Close'].iloc[i] > positive_variation * price_bought:
		portfolio['bank'].iloc[i+1] = portfolio['bank'].iloc[i] + portfolio['Close'].iloc[i] * portfolio['stocks_owned'].iloc[i]
		portfolio['stocks_owned'].iloc[i+1] = 0
		portfolio['position'].iloc[i+1] = 'Sold'
		print("Vendeu por {}".format(portfolio['Close'].iloc[i]))
	else:
		portfolio['position'].iloc[i+1] = portfolio['position'].iloc[i]
		portfolio['bank'].iloc[i+1] = portfolio['bank'].iloc[i]
		portfolio['stocks_owned'].iloc[i+1] = portfolio['stocks_owned'].iloc[i]
		print("Não vendeu.")

portfolio['Total$'] = portfolio['stocks_owned'] * portfolio['Close'] + portfolio['bank']
portfolio['Total$'].describe()

Comprou por 17.770000457763672
Não vendeu.
Não vendeu.
Não vendeu.
Não vendeu.


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


Não vendeu.
Não vendeu.
Não vendeu.
Não vendeu.
Não vendeu.
Não vendeu.
Não vendeu.
Não vendeu.
Não vendeu.
Não vendeu.
Não vendeu.
Não vendeu.
Não vendeu.
Não vendeu.
Não vendeu.
Não vendeu.
Não vendeu.
Não vendeu.
Não vendeu.
Não vendeu.
Não vendeu.
Não vendeu.
Não vendeu.
Não vendeu.
Não vendeu.
Não vendeu.
Não vendeu.
Não vendeu.
Não vendeu.
Não vendeu.
Não vendeu.
Não vendeu.
Não vendeu.
Não vendeu.
Não vendeu.
Não vendeu.
Não vendeu.
Não vendeu.
Não vendeu.
Não vendeu.
Não vendeu.
Vendeu por 18.149999618530273
Não comprou
Não comprou
Não comprou
Não comprou
Não comprou
Comprou por 18.170000076293945
Não vendeu.
Não vendeu.
Não vendeu.
Não vendeu.
Não vendeu.
Não vendeu.
Não vendeu.
Não vendeu.
Não vendeu.
Não vendeu.
Não vendeu.
Não vendeu.
Não vendeu.
Não vendeu.
Não vendeu.
Não vendeu.
Não vendeu.
Não vendeu.
Não vendeu.
Não vendeu.
Não vendeu.
Não vendeu.
Não vendeu.
Não vendeu.
Não vendeu.
Não vendeu.
Não vendeu.


count       81.000000
mean     10167.628257
std        114.017357
min       9955.040043
25%      10073.059528
50%      10134.879871
75%      10236.038971
max      10376.538971
Name: Total$, dtype: float64

In [ ]:
portfolio.tail()

,position,stocks_owned,bank,Model Recommendation,Close,Total$
Datetime,,,,,,
2020-04-29 16:30:00-03:00,Bought,562,2.019485,0,18.430000,10359.679657
2020-04-29 16:35:00-03:00,Bought,562,2.019485,1,18.410000,10348.439400
2020-04-29 16:40:00-03:00,Bought,562,2.019485,0,18.440001,10365.299786
2020-04-29 16:45:00-03:00,Bought,562,2.019485,0,18.400000,10342.819271
2020-04-29 16:50:00-03:00,Bought,562,2.019485,0,18.280001,10275.379871


# ALL TOGETHER

In [ ]:
day_to_analyze = '2020-10-18' + '10:00:00-03:00'
day_to_analyze[:10]

'2020-10-18'

In [2]:
!pip install yfinance
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report,confusion_matrix
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping
from xgboost import XGBClassifier

     |████████████████████████████████| 5.5MB 6.3MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.55-py2.py3-none-any.whl size=22618 sha256=1732aee0c54c9db028357873c3d8be07f3d664bb839079ea30752c3ca264ad4c
  Stored in directory: /root/.cache/pip/wheels/04/98/cc/2702a4242d60bdc14f48b4557c427ded1fe92aedf257d4565c
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [ ]:
day_to_analyze = '2020-10-18'
day_start = day_to_analyze + ' 10:00:00-03:00'
day_end = day_to_analyze + ' 16:50:00-03:00'
df['2020-10-18':]

,Close,Volume
Datetime,,
2020-10-19 10:12:00-03:00,19.410000,200200
2020-10-19 10:14:00-03:00,19.410000,372000
2020-10-19 10:16:00-03:00,19.389999,203800
2020-10-19 10:18:00-03:00,19.440001,146200
2020-10-19 10:20:00-03:00,19.450001,245600
...,...,...
2020-10-19 15:50:00-03:00,19.480000,573100
2020-10-19 15:52:00-03:00,19.540001,1264400
2020-10-19 15:54:00-03:00,19.570000,349400


In [12]:
import pandas as pd
import numpy as np
from tensorflow import keras

model = keras.models.load_model(r'..\ml-financial-market\models')
df = pd.read_csv(r'..\ml-financial-market\data\interim\interim.csv')
X_test = pd.read_csv(r'..\ml-financial-market\data\processed\X_test.csv')


In [13]:
X_test

,0,1,2,3,4,5,6,7,8,9,...,47,48,49,50,51,52,53,54,55,56
0,0.888528,0.324723,0.005659,0.312618,0.299435,0.290019,0.296296,0.292279,0.299632,0.325368,...,0.021351,0.022229,0.078788,0.007840,0.003329,0.041603,0.002735,0.006445,0.002120,0.005594
1,0.769481,0.531365,0.002361,0.519774,0.512241,0.514124,0.525926,0.533088,0.540441,0.542279,...,0.005848,0.009464,0.041349,0.012251,0.006087,0.047783,0.013601,0.028259,0.084382,0.029185
2,0.635714,0.422509,0.004398,0.416196,0.419962,0.419962,0.440741,0.444853,0.426470,0.419118,...,0.009155,0.009256,0.031344,0.014176,0.011763,0.045022,0.007379,0.007143,0.007333,0.015222
3,0.111255,0.804428,0.002106,0.809793,0.806026,0.815443,0.827778,0.825368,0.816176,0.821691,...,0.001804,0.002151,0.009993,0.007766,0.005319,0.009803,0.001828,0.002289,0.004099,0.002917
4,0.063203,0.952029,0.001683,0.947269,0.947269,0.947269,0.951852,0.943014,0.948529,0.950368,...,0.002184,0.001764,0.012939,0.005209,0.001135,0.014566,0.001797,0.003320,0.001951,0.001378
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1382,0.009307,0.970480,0.007940,0.966102,0.956686,0.954802,0.955556,0.952206,0.955882,0.950368,...,0.002476,0.001457,0.028630,0.002387,0.001075,0.008450,0.001458,0.002027,0.003860,0.003372
1383,0.140260,0.824723,0.003043,0.821092,0.832392,0.828625,0.835185,0.830882,0.825368,0.816176,...,0.004672,0.013502,0.028030,0.007621,0.005439,0.058367,0.004456,0.005220,0.003576,0.004553
1384,0.447835,0.490775,0.003102,0.485876,0.478343,0.478343,0.505556,0.507353,0.520221,0.518382,...,0.023618,0.011541,0.041193,0.009523,0.007528,0.222584,0.002544,0.002519,0.004595,0.004220
1385,0.490260,0.369004,0.002529,0.355932,0.357815,0.369115,0.383333,0.362132,0.367647,0.356618,...,0.003950,0.008073,0.045906,0.004543,0.006435,0.062070,0.004466,0.006184,0.002412,0.007629


In [4]:
data = yf.download("PETR4.SA", period='30d',  interval = "2m")

df = data
df = df.drop(['High', 'Low', 'Adj Close', 'Open'],axis=1)

# Creating t-n columns for Close
for i in range(1,28):
    df['Close_Minus{}'.format(i)] = df['Close'].shift(i)

# Creating t-n columns for Volume    
for i in range(1,28):
    df['Volume_Minus{}'.format(i)] = df['Volume'].shift(i)

day_to_analyze = '2020-10-19'
day_start = day_to_analyze + ' 10:00:00-03:00'
day_end = day_to_analyze + ' 16:50:00-03:00'

df = df.loc[day_start:]

scaler = MinMaxScaler()
df = scaler.fit_transform(df)


[*********************100%***********************]  1 of 1 completed


In [11]:
data = yf.download("PETR4.SA", period='30d',  interval = "2m")

df = data
df = df.drop(['High', 'Low', 'Adj Close', 'Open'],axis=1)

# Creating t-n columns for Close
for i in range(1,28):
    df['Close_Minus{}'.format(i)] = df['Close'].shift(i)

# Creating t-n columns for Volume    
for i in range(1,28):
    df['Volume_Minus{}'.format(i)] = df['Volume'].shift(i)

day_to_analyze = '2020-10-19'
day_start = day_to_analyze + ' 10:00:00-03:00'
day_end = day_to_analyze + ' 16:50:00-03:00'

df = df.loc[day_start:]

scaler = MinMaxScaler()
df = scaler.fit_transform(df)

# Feature engineering
bvsp = data
bvsp = bvsp[['Close', 'Volume']]

# Creating t-n columns for Close
for i in range(1,28):
    bvsp['Close_Minus{}'.format(i)] = bvsp['Close'].shift(i)

# Creating t-n columns for Volume    
for i in range(1,28):
    bvsp['Volume_Minus{}'.format(i)] = bvsp['Volume'].shift(i)

bvsp = bvsp.loc[:day_to_analyze[:10] + ' 16:50:00-03:00']

bvsp_ann = bvsp.dropna() # dropping NAs

bvsp_ann['Rise'] = 1
periods_ahead = 180 # 1 and a half hours
positive_variation = 1.02

for i in range(1,len(bvsp_ann)-periods_ahead):
    for j in range(1,periods_ahead):
        if bvsp_ann['Close'].iloc[i]/bvsp_ann['Close'].iloc[i+j] > positive_variation:
            bvsp_ann['Rise'].iloc[i] = 1
            break
        else:
            bvsp_ann['Rise'].iloc[i] = 0

bvsp_ann = bvsp_ann.iloc[:-periods_ahead]


# Splitting into subsets
X = bvsp_ann.drop('Rise', axis=1)
y = bvsp_ann['Rise']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)

# Scaling data
scaler = MinMaxScaler()
X_train= scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Creating Early Stop callback
early_stop = EarlyStopping(monitor='val_loss', mode='auto', verbose=1, patience=15)

# Building the model
model = Sequential()
model.add(Dense(units=28,activation='relu'))
model.add(Dense(units=22,activation='relu'))
model.add(Dense(units=15,activation='relu'))
model.add(Dense(units=10,activation='relu'))
model.add(Dense(units=1,activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
             metrics=['accuracy'])

# Fitting the model
model.fit(x=X_train, 
          y=y_train.values,
          epochs=2000, verbose=1,
         validation_data=(X_train,y_train.values),
         callbacks=[early_stop])

portfolio = pd.DataFrame(columns=['position', 'stocks_owned', 'bank'])
portfolio['position'] = 'Sold'
portfolio['bank'] = 10000
portfolio['stocks_owned'] = 0
portfolio['Model Recommendation'] = 0
portfolio['Close'] = bvsp_ann['Close']
portfolio.head()

price_bought = 0

for i in range(len(portfolio)-1):
	buy_or_not = model.predict_classes(np.array([df[i],]))[0][0]
	portfolio['Model Recommendation'].iloc[i] = buy_or_not
	if portfolio['position'].iloc[i] == 'Sold':
		if buy_or_not == 1:
			portfolio['stocks_owned'].iloc[i+1] = int(portfolio['bank'].iloc[i]/portfolio['Close'].iloc[i])
			portfolio['bank'].iloc[i+1] = (portfolio['bank'].iloc[i] % portfolio['Close'].iloc[i])
			portfolio['position'].iloc[i+1] = 'Bought'
			price_bought = portfolio['Close'].iloc[i]
			print('Comprou por {}'.format(price_bought))
		else:
			portfolio['position'].iloc[i+1] = portfolio['position'].iloc[i]
			portfolio['bank'].iloc[i+1] = portfolio['bank'].iloc[i]
			portfolio['stocks_owned'].iloc[i+1] = portfolio['stocks_owned'].iloc[i]
			print('Não comprou')
	elif portfolio['Close'].iloc[i] < (0.95) * price_bought:
		portfolio['bank'].iloc[i+1] = portfolio['bank'].iloc[i] + portfolio['Close'].iloc[i] * portfolio['stocks_owned'].iloc[i]
		portfolio['stocks_owned'].iloc[i+1] = 0
		portfolio['position'].iloc[i+1] = 'Sold'
		print("Vendeu por {}".format(portfolio['Close'].iloc[i]))
	elif portfolio['Close'].iloc[i] > positive_variation * price_bought:
		portfolio['bank'].iloc[i+1] = portfolio['bank'].iloc[i] + portfolio['Close'].iloc[i] * portfolio['stocks_owned'].iloc[i]
		portfolio['stocks_owned'].iloc[i+1] = 0
		portfolio['position'].iloc[i+1] = 'Sold'
		print("Vendeu por {}".format(portfolio['Close'].iloc[i]))
	else:
		portfolio['position'].iloc[i+1] = portfolio['position'].iloc[i]
		portfolio['bank'].iloc[i+1] = portfolio['bank'].iloc[i]
		portfolio['stocks_owned'].iloc[i+1] = portfolio['stocks_owned'].iloc[i]
		print("Não vendeu.")

portfolio['Total$'] = portfolio['stocks_owned'] * portfolio['Close'] + portfolio['bank']
portfolio['Total$'].describe()

portfolio.tail()

[*********************100%***********************]  1 of 1 completed


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-

Epoch 1/2000
89/89 [==============================] - 0s 3ms/step - loss: 0.6848 - accuracy: 0.6365 - val_loss: 0.6747 - val_accuracy: 0.6510
Epoch 2/2000
89/89 [==============================] - 0s 2ms/step - loss: 0.6549 - accuracy: 0.6599 - val_loss: 0.6234 - val_accuracy: 0.6929
Epoch 3/2000
89/89 [==============================] - 0s 2ms/step - loss: 0.5989 - accuracy: 0.6645 - val_loss: 0.5807 - val_accuracy: 0.6869
Epoch 4/2000
89/89 [==============================] - 0s 2ms/step - loss: 0.5753 - accuracy: 0.6695 - val_loss: 0.5705 - val_accuracy: 0.6606
Epoch 5/2000
89/89 [==============================] - 0s 2ms/step - loss: 0.5721 - accuracy: 0.6720 - val_loss: 0.5656 - val_accuracy: 0.6901
Epoch 6/2000
89/89 [==============================] - 0s 2ms/step - loss: 0.5736 - accuracy: 0.6826 - val_loss: 0.5767 - val_accuracy: 0.7359
Epoch 7/2000
89/89 [==============================] - 0s 2ms/step - loss: 0.5641 - accuracy: 0.6716 - val_loss: 0.5611 - val_accuracy: 0.6624
Epoch 

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


ValueError: ignored